# Sentiment Classification & Embedding II

* Embedding Layer
* Sequence Model

# 01. What data we use?

In [41]:
import numpy as np
import pandas as pd

In [42]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/steam.txt'
data= pd.read_table(url, names=['label', 'reviews'])

In [43]:
data.head()

,label,reviews
0,0,노래가 너무 적음
1,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
2,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
3,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
4,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요


In [44]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=2021)
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [45]:
train_data.shape, test_data.shape

((80000, 2), (20000, 2))

In [46]:
test_data.head()

,label,reviews
0,0,절대로 사지마세요 님들아이거 사서 열심히 설치해서 해봤는데 멀티밖에 안되는 게임이 ...
1,1,조금 해봤는데 재미있네요
2,1,제작자가 행동 하나하나를 다 꿰뚫고있음 뭔 병신짓을 해도 다 받아줘서 좋음 10/10
3,1,길이길이 남을 대작 게임.설명하려 들고 싶지 않을 정도로 완벽 그자체. 꼭 해보세요!
4,0,한글문제인건지 실행은되나 리그 시작 자체가 안됨. 돈받고 파는거면 적어도 어떤 환경...


In [47]:
train_data.tail()

,label,reviews
79995,1,"재밌게 플레이하는 중! 하지만, 밸런스가 업데이트에 따라 엄청나게 바뀌고 DLC 장..."
79996,1,네이버에서 한글패치를 찾아 적용해 플레이 해보길바란다.. 포탈의 스토리를 좋아한다면...
79997,0,실행 자체가 안된다. 업데이트 버튼에서 멈춰있는데 무슨 평가를 하란거냐? 저 0.6...
79998,1,심판 가정교육 독학했나 판정 뭐같이하네
79999,1,스토리도 좋고 이제 3편만...


1 0이 각각 긍/부정 라벨링을 하는 것 같다. 

# 한국어 기본 전처리

In [48]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
train_data.isna().sum()

label      0
reviews    0
dtype: int64

In [50]:
# 결측치 제거
trian_data = train_data.dropna(how='any')
# 한글과 공백을 제외하고 모두 제거
train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", " ")
train_data[:5]

<ipython-input-50-b72c8d08a91c>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", " ")


,label,reviews
0,0,스토리도 별루고 버그도 많고 잼없네요
1,1,조아용
2,0,지금도 유저들 있음
3,1,멀티가 안되서 아쉽긴 하지만 재미 있어여
4,0,난이도는 상관 안한다 뒤지면서 배우면 되니까 근데 업데이트 안해주는건 ㄹㅇ 개빡친다


In [51]:
import konlpy
from konlpy.tag import Okt
okt = Okt()
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

In [52]:
%%time

# x_train 토큰화, 불용어 제거 전처리 수행
x_train = []
for sentence in train_data['reviews']:
    temp_x = []
    temp_x = okt.morphs(sentence, stem = True) # 토큰화
    temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
    x_train.append(temp_x)

CPU times: user 5min 55s, sys: 1.63 s, total: 5min 57s
Wall time: 6min 14s


In [53]:
print(x_train[:4])

[['스토리', '별', '루고', '버그', '많다', '잼', '없다'], ['좋다'], ['지금', '유저', '있다'], ['멀티', '안되다', '아쉽다', '하지만', '재미', '있다']]


In [54]:
%%time

# Null 값 제거
test_data = test_data.dropna(how = 'any')
# 정규 표현식 수행
test_data['reviews'] = test_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", " ")

# x_test 토큰화, 불용어 제거 전처리 수행
x_test = []
for sentence in test_data['reviews']:
    temp_x = []
    temp_x = okt.morphs(sentence, stem = True) # 토큰화
    temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
    x_test.append(temp_x)

<timed exec>:4: FutureWarning: The default value of regex will change from True to False in a future version.


CPU times: user 1min 33s, sys: 450 ms, total: 1min 33s
Wall time: 1min 39s


In [55]:
print(x_test[:5])

[['절대로', '사지', '말다', '님', '아이거', '사서', '열심히', '설치', '해봤다', '멀티', '밖에', '안되다', '게임', '멀티', '들어가다', '핑높다', '고', '강퇴', '당하다'], ['조금', '해봤다', '재미있다'], ['제작자', '행동', '하나', '하나', '다', '꿰뚫다', '뭔', '병신', '짓', '을', '해도', '다', '받다', '좋다'], ['길이', '길이', '남', '을', '대작', '게임', '설명', '들다', '싶다', '않다', '정도', '로', '완벽', '그', '자체', '꼭', '해보다'], ['한글', '문제', '인', '건지다', '실행', '되다', '리그', '시작', '자체', '안되다', '돈', '받다', '팔다', '적어도', '어떻다', '환경', '이든', '실행', '되게', '금', '않다']]


# Q1. Tokenizing & Text to Sequences

* 상위 40000개 단어만 사용한다.

In [56]:
## 위의 한국어 전처리를 했다면 사용하지 않는다.
# 명확히 str로 데이터 타입을 변경해 줘야 tokenizer에서 에러가 없다. 한국어 건들 때만 그러는 것 같음.
x_train = train_data['reviews'].astype('str').tolist()
x_test = test_data['reviews'].astype('str').tolist()

In [57]:
x_train[:5]

['스토리도 별루고 버그도 많고 잼없네요  ',
 '조아용',
 '지금도 유저들 있음 ',
 '멀티가 안되서 아쉽긴 하지만 재미 있어여',
 '난이도는 상관 안한다  뒤지면서 배우면 되니까  근데 업데이트 안해주는건 ㄹㅇ 개빡친다']

In [58]:
# 라벨로 넘긴다.
y_train = train_data['label'].values
y_test = test_data['label'].values

In [59]:
### Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
max_words = 40000 # 상위 40,000개의 단어만 보존 
tokenizer = Tokenizer(num_words = max_words)

In [60]:
### Tokenizer
%%time
# tokenizer token : idx 토크나이즈하고 인덱스 다 붙이는 것. 과정이 매칭되어야 한다.
tokenizer.fit_on_texts(x_train)
# Text --> Sequence, text seq > idx seq 매칭
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

CPU times: user 4.61 s, sys: 53 ms, total: 4.67 s
Wall time: 4.37 s


# Padding Sequence

* 문장의 최대 길이는 40으로 한다.

In [61]:
max_words = max_words # 위에서 설정한 40000
embedding_dim = 128 # 단어 embedding 차원
max_len = 40 # 문장 최대 길이

In [62]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [63]:
#### Pad Sequences > 패드 시퀀스가 뭐죠? 문장 길이를 통일하는 겁니다. 단, 사용자 편의성을 위해서 패딩과 트리밍이 둘 다 있슴.
x_train = pad_sequences(x_train, maxlen = max_len)
x_test = pad_sequences(x_test, maxlen = max_len)

In [64]:
x_train[:3]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
          210, 19202,   666,   320],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 32932],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,  1335,  3378,    76]], dtype=int32)

In [65]:
x_train[2]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 1335, 3378,   76], dtype=int32)

In [66]:
train_data.iloc[2]

label                0
reviews    지금도 유저들 있음 
Name: 2, dtype: object

In [67]:
x_train = np.array(x_train)
x_test = np.array(x_test)

# 0으로 가득찬 데이터에 대한 추가 전처리

In [68]:
not0train_idx = x_train.sum(1) > 0
not0test_idx = x_test.sum(1) > 0

In [69]:
train_data = train_data.loc[not0train_idx].reset_index(drop=True)
test_data = test_data.loc[not0test_idx].reset_index(drop=True)

In [75]:
train_data.tail()

,label,reviews
77151,1,재밌게 플레이하는 중 하지만 밸런스가 업데이트에 따라 엄청나게 바뀌고 장...
77152,1,네이버에서 한글패치를 찾아 적용해 플레이 해보길바란다 포탈의 스토리를 좋아한다면...
77153,0,실행 자체가 안된다 업데이트 버튼에서 멈춰있는데 무슨 평가를 하란거냐 저 ...
77154,1,심판 가정교육 독학했나 판정 뭐같이하네
77155,1,스토리도 좋고 이제 편만


In [70]:
x_train = x_train[not0train_idx]
y_train = y_train[not0train_idx]

x_test = x_test[not0test_idx]
y_test = y_test[not0test_idx]

In [79]:
x_train[:3]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
          210, 19202,   666,   320],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 32932],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,  1335,  3378,    76]], dtype=int32)

# 모델링

**모델 구조**
1. 임베딩 레이어 : 임베딩차원은 128
2. Conv1D 블록 : 필터수 64개, 윈도우 사이즈 5
3. Bidirectional layer :
    * 정방향 : LSTM, 히든스테이트 32 
    * 역방향 : LSTM, 히든스테이트 32
4. Bidirectional layer :
    * 정방향 : GRU, 히든스테이트 32
    * 역방향 : RNN, 히든스테이트 16
5. Conv1D 블록 : 필터수 32개, 윈도우 사이즈 5
6. MaxPool1D 블록 : 필터사이즈2
7. 플래튼
8. FC Layer : 노드 1024개
9. 시그모이드 레이어

In [ ]:
#####################
## your codes here ##
#####################





In [ ]:
#####################
## your codes here ##
#####################






# EarlyStopping을 이용한 학습.

1. 20%는 벨리데이션 셋.
2. 4epochs전과 비교하여 early stopping할 것.

In [ ]:
#####################
## your codes here ##
#####################




In [ ]:
#####################
## your codes here ##
#####################






In [ ]:
review_idx = 128

temp = test_data.loc[review_idx]
docu = temp['reviews']
label = 'positive' if temp['label'] ==1 else 'Negative'

print(f"문서 번호 {review_idx}")
print(label, " : ", docu)
y_pred = model.predict(x_test[review_idx:review_idx+1])
label_pred = 'positive' if y_pred[0,0] >=0.5 else 'Negative'
print(f"모델의 예측 : {label_pred},   prob = {y_pred[0,0]*100:.2f}%")